<a href="https://colab.research.google.com/github/josepmdc/ApC-PLAB1/blob/master/PLAB1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>